# ehrdata and the physionet2012 challenge dataset

This is a small notebook instructing how

1. To install `ehrdata`
2. Install the required ehrapy package "development install" from the development branch on which ehrapy starts to support time-series operations
3. How `ehrdata` can download and prepare the Physionet2012 Challenge dataset out of the box
4. How a small sequence model can be trained

Resources

- Physionet2012 Challenge Paper: https://ieeexplore.ieee.org/abstract/document/6420376
- Data Link: https://physionet.org/content/challenge-2012/1.0.0/


## 1. Install ehrdata

Install a development version:

In [39]:
!pip install "ehrdata==0.0.2a1"

## 2. Install ehrapy
The ehrapy package does not yet support time-series operations, as this is a feature we're developing.

But the latest developments can be used by executing the line below. This fetches the ehrapy repository, switches to the time-series branch, and installs the ehrapy package from this branch.

In [40]:
!git clone https://github.com/theislab/ehrapy.git
!cd ehrapy && git fetch && git checkout feature/time-series && pip install -e .

fatal: destination path 'ehrapy' already exists and is not an empty directory.
Already on 'feature/time-series'
Your branch is behind 'origin/feature/time-series' by 3 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Obtaining file:///Users/elifguvercin/ehrapy
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... one
  Preparing editable metadata (pyproject.toml) ... done
  Using cached numpy-2.2.4-cp312-cp312-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_x86_64.whl.metadata (62 kB)
Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_x86_64.whl (6.6 MB)
  Building editable for ehrapy (pyproject.toml) ... done
  Created wheel for ehrapy: filename=ehrapy-0.12.0-py3-none-any.whl size=10899 sha256=67b89d54741ac58d6e49df05fbd647e46c343c2d1100e3ab40bb48bec8e194fe
  Stored in directory

**IMPORTANT** You need to restart your Jupyter kernel after running the ehrapy dev install; Only then will juptyer recognize the freshly installed ehrapy package.

Once you restarted, you don't need to run the two "installations" above, but can continue directly below here.

## 3. Prepare the Physionet2012 Challenge Dataset

### 3.1 Imports

In [3]:
import numpy as np
print(np.__version__)

2.1.3


In [41]:
pip install "numpy==1.26.4"

  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ehrapy 0.12.0 requires numpy>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
print(np.__version__)

1.26.4


In [2]:
import numpy as np
from pathlib import Path
import ehrdata as ed

### 3.2 Data Loading
The physionet2012 challenge dataset is one of the ready-to-use datasets of `ehrdata`. See the documentation for further information.

In [3]:
edata = ed.dt.physionet2012()

/opt/anaconda3/envs/goodmimic/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/anaconda3/envs/goodmimic/lib/python3.12/site-packages/ehrdata/core/ehrdata.py:117: ImplicitModificationWarning: Setting element `.layers['r_layer']` of view, initializing view as actual.
  self.layers[R_LAYER_KEY] = input


The `edata` object contains the data in different fields. The most important ones are

- `edata.r` is 3D n_obs x n_var x n_timesteps numpy array containing the time series data
- `edata.obs` is a dataframe containing static information about the individuals
- `edata.var` is a dataframe which can contain information on the features; here, its quite boring and just the names
- `edata.t` is a dataframe which can contain information about the timeaxis; here, its quite boring and just the number of the hours


In [ ]:
edata

In [ ]:
edata.r.shape

In [ ]:
edata.r.copy()

In [ ]:
edata.obs['In-hospital_death'].value_counts()

In [ ]:
edata.var

In [4]:
# one drop and make it 36 features
edata = edata[:, edata.var_names != "MechVent"] 

/opt/anaconda3/envs/goodmimic/lib/python3.12/site-packages/ehrdata/core/ehrdata.py:117: ImplicitModificationWarning: Setting element `.layers['r_layer']` of view, initializing view as actual.
  self.layers[R_LAYER_KEY] = input


In [5]:
edata.var.shape,edata.t.shape

((36, 1), (48, 1))

### 4. Training a Sequence Model

#### 4.1 Data Preprocessing

This shows how the data can be split, prepared into a pytorch dataloader, and be used for a small pytorch model. The model itself is not competitive or "good", but is just a small example.

We split the data into train, validation, and test splits, using the "set" labels provided in the Physionet2012 challenge dataset. Any other split is also possible.

In [6]:
edata_train = edata[edata.obs["set"] == "set-a"]
edata_val = edata[edata.obs["set"] == "set-b"]
edata_test = edata[edata.obs["set"] == "set-c"]

/opt/anaconda3/envs/goodmimic/lib/python3.12/site-packages/ehrdata/core/ehrdata.py:117: ImplicitModificationWarning: Setting element `.layers['r_layer']` of view, initializing view as actual.
  self.layers[R_LAYER_KEY] = input
/opt/anaconda3/envs/goodmimic/lib/python3.12/site-packages/ehrdata/core/ehrdata.py:117: ImplicitModificationWarning: Setting element `.layers['r_layer']` of view, initializing view as actual.
  self.layers[R_LAYER_KEY] = input
/opt/anaconda3/envs/goodmimic/lib/python3.12/site-packages/ehrdata/core/ehrdata.py:117: ImplicitModificationWarning: Setting element `.layers['r_layer']` of view, initializing view as actual.
  self.layers[R_LAYER_KEY] = input



We perform a zero-mean and unit-variance normalization. For this, ehrapy provides some functionality:

In [7]:
import ehrapy as ep

In [8]:
scaler3d = ep.ts.StandardScaler3D()

scaler3d.fit_transform(edata_train)
scaler3d.transform(edata_val)
scaler3d.transform(edata_test)

To impute missing values, we now perform last observation carried forward imputation.
There exist also more sophisticated methods for time-series imputation, e.g. [SAITS (Du et al., 2023)](https://doi.org/10.1016/j.eswa.2023.119619).

In [9]:
locf = ep.ts.LOCFImputer()

locf.fit_transform(edata_train)
locf.transform(edata_val)
locf.transform(edata_test)

Finally, let us briefly explore the shape of the data: We have 37 numeric features, measured for 48 hours, and roughly 4'000 patients per split. The dataset is imbalanced, with ca 14% in-hospital death cases per split.

In [ ]:
x

In [10]:
edata

EHRData object with n_obs x n_var = 11988 x 36, and a timeseries of 48 steps.
             shape of .X: (0, 0) 
             shape of .r: (11988, 36, 48) 

In [11]:
edata_val

EHRData object with n_obs x n_var = 3993 x 36, and a timeseries of 48 steps.
             shape of .X: (0, 0) 
             shape of .r: (3993, 36, 48) 

In [12]:
edata_test

EHRData object with n_obs x n_var = 3998 x 36, and a timeseries of 48 steps.
             shape of .X: (0, 0) 
             shape of .r: (3998, 36, 48) 

In [ ]:
print(
    f"positive rate in edata_train: {edata_train.obs['In-hospital_death'].mean():.4f}, absolute count: {edata_train.obs['In-hospital_death'].sum()}"
)

print(
    f"positive rate in edata_val: {edata_val.obs['In-hospital_death'].mean():.4f}, absolute count: {edata_val.obs['In-hospital_death'].sum()}"
)

print(
    f"positive rate in edata_test: {edata_test.obs['In-hospital_death'].mean():.4f}, absolute count: {edata_test.obs['In-hospital_death'].sum()}"
)

#### 4.2 Model preparation

The below part is not very specific for `ehrdata` or `ehrapyz` anymore, but more about `torch`.

It just is a tiny showcase for a sequence model.

We build a small sequence model in pytorch. More elaborate model architectures have been proposed for this dataset. Here, for demonstration purposes and as to avoid complexity, we focus on a simple architecture.

In a nutshell, the model has a GRU block as sequence model, and a classification head.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary
from sklearn.metrics import roc_auc_score, precision_recall_curve, accuracy_score, f1_score, auc, confusion_matrix


In [34]:
# Step 1: Reorder edata.r based on the random 6x6 grid
flat_order = [
    'FiO2', 'HCO3', 'Bilirubin', 'SaO2', 'Urine', 'NIDiasABP',
    'TroponinI', 'Platelets', 'Na', 'RespRate', 'TroponinT', 'GCS',
    'Glucose', 'SysABP', 'DiasABP', 'NIMAP', 'Lactate', 'pH',
    'Cholesterol', 'PaO2', 'ALP', 'Weight', 'HCT', 'Mg',
    'AST', 'WBC', 'Temp', 'NISysABP', 'Albumin', 'PaCO2',
    'BUN', 'HR', 'K', 'MAP', 'ALT', 'Creatinine'
]

# Get feature indices from edata.var
indices = [edata.var.index.get_loc(f) for f in flat_order]

# Reorder time series data in edata.r
edata_train.r = edata_train.r[:, indices, :]
edata_val.r = edata_val.r[:, indices, :]
edata_test.r = edata_test.r[:, indices, :]

In [42]:
def reshape_to_6x6_tensor(x):
    # x shape: (n_samples, 36, 48)
    x = x.transpose(0, 2, 1)  # → (n_samples, 48, 36)
    x = x.reshape(x.shape[0], x.shape[1], 6, 6)  # → (n_samples, 48, 6, 6)
    x = x[:, :, None, :, :]  # → (n_samples, 48, 1, 6, 6)
    return x


In [32]:
y_train = edata_train.obs["In-hospital_death"].values
y_val = edata_val.obs["In-hospital_death"].values
y_test = edata_test.obs["In-hospital_death"].values

In [41]:
# 2. Convert to tensors
trn_ds = TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).long())
val_ds = TensorDataset(torch.tensor(x_val).float(), torch.tensor(y_val).long())
tst_ds = TensorDataset(torch.tensor(x_test).float(), torch.tensor(y_test).long())

In [43]:
x_train = reshape_to_6x6_tensor(edata_train.r)
x_val = reshape_to_6x6_tensor(edata_val.r)
x_test = reshape_to_6x6_tensor(edata_test.r)

In [61]:
x_train.shape

In [62]:
# 3. Create data loaders
trn_dl = DataLoader(trn_ds, batch_size=256, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=256, shuffle=False)
tst_dl = DataLoader(tst_ds, batch_size=256, shuffle=False)

In [63]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [64]:
class CostumConv3d(nn.Module):
    def __init__(self, ni, no, kernel, stride, pad, drop=None, activ=lambda: nn.ReLU(inplace=True)):
        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [nn.Conv3d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout3d(drop))  # 3D Dropout for consistency
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


class Flatten(nn.Module):
    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)


class Classifier(nn.Module):
    def __init__(self, raw_ni=None, no=2, drop=0.5):
        super().__init__()

        self.conv_block = nn.Sequential(
            CostumConv3d(1, 16, kernel=(3, 3, 3), stride=1, pad=1, drop=drop),
            CostumConv3d(16, 32, kernel=(3, 3, 3), stride=1, pad=1, drop=drop),
            nn.AdaptiveAvgPool3d((1, 1, 1))  # → (batch, 32, 1, 1, 1)
        )

        self.flatten = Flatten()

        self.fc = nn.Sequential(
            nn.Dropout(drop),
            nn.Linear(32, 64),  # Output of conv block is (batch, 32)
            nn.ReLU(inplace=True),
            nn.Dropout(drop),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, no)
        )

    def forward(self, t_raw):
        # t_raw shape: (batch, 1, 48, 6, 6)
        x = self.conv_block(t_raw)  # → (batch, 32, 1, 1, 1)
        x = self.flatten(x)         # → (batch, 32)
        return self.fc(x)


        
"""class Classifier(nn.Module):
    def __init__(self, raw_ni, no, input_time_steps=48, drop=0.5):
        super().__init__()

        # First and second conv layer
        self.conv_block = nn.Sequential(
            CostumConv1d(raw_ni, 64, kernel=3, stride=1, pad=0, drop=drop),
            CostumConv1d(64, 128, kernel=3, stride=1, pad=0, drop=drop),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.flatten = Flatten()

        # Dynamically calculate flattened size
        with torch.no_grad():
            dummy_input = torch.zeros(1, raw_ni, input_time_steps)
            dummy_output = self.conv_block(dummy_input)
            flat_dim = dummy_output.view(1, -1).shape[1]

        self.fc = nn.Sequential(
            nn.Dropout(drop), nn.Linear(flat_dim, 64), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True),
            nn.Linear(64, no)
        )

    def forward(self, t_raw):
        x = self.conv_block(t_raw)
        x = self.flatten(x)
        return self.fc(x)"""


'class Classifier(nn.Module):\n    def __init__(self, raw_ni, no, input_time_steps=48, drop=0.5):\n        super().__init__()\n\n        # First and second conv layer\n        self.conv_block = nn.Sequential(\n            CostumConv1d(raw_ni, 64, kernel=3, stride=1, pad=0, drop=drop),\n            CostumConv1d(64, 128, kernel=3, stride=1, pad=0, drop=drop),\n            nn.MaxPool1d(kernel_size=2, stride=2)\n        )\n        self.flatten = Flatten()\n\n        # Dynamically calculate flattened size\n        with torch.no_grad():\n            dummy_input = torch.zeros(1, raw_ni, input_time_steps)\n            dummy_output = self.conv_block(dummy_input)\n            flat_dim = dummy_output.view(1, -1).shape[1]\n\n        self.fc = nn.Sequential(\n            nn.Dropout(drop), nn.Linear(flat_dim, 64), nn.ReLU(inplace=True),\n            nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True),\n            nn.Linear(64, no)\n        )\n\n    def forward(self, t_raw):\n        x = self

In [65]:
for epoch in range(1):
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        x_raw, y_batch = [t.to(device) for t in batch]
x_raw.shape,y_batch.shape


(torch.Size([157, 48, 1, 6, 6]), torch.Size([157]))

In [66]:
x_raw=x_raw.permute(0, 2, 1, 3, 4)
x_raw.shape

torch.Size([157, 1, 48, 6, 6])

In [67]:
#raw_feat = x_train.shape[1]  # 37 features = 37 input channels
num_classes = 2
drop = 0.3
loss_history = []
acc_history = []
patience, trials = 200, 0
base, step = 1, 2
best_acc = 0
n_epochs = 100

model = Classifier(no=num_classes, drop=drop).to(device)
#criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.5]).to(device))#Compares predicted logits (before softmax) to true labels
opt = optim.Adam(model.parameters(), lr=0.001)

# Example pass-through for shape check
x_dummy = torch.randn(1,1,48,6,6).to(device)
print("Dummy input shape:", x_dummy.shape)
with torch.no_grad():
    print("Output shape:", model(x_dummy).shape)

model = Classifier(no=num_classes, drop=drop).to(device)
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001)

print('Start model training')



Dummy input shape: torch.Size([1, 1, 48, 6, 6])
Output shape: torch.Size([1, 2])
Start model training


In [68]:
for epoch in range(1, n_epochs + 1):
    model.train()
    epoch_loss = 0
    for x_raw, y_batch in trn_dl:
        x_raw, y_batch = x_raw.to(device), y_batch.to(device)
        opt.zero_grad()
        out = model(x_raw)
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()

    epoch_loss /= len(trn_dl)
    loss_history.append(epoch_loss)

    # Validation
    model.eval()
    correct, total = 0, 0
    for x_raw, y_batch in val_dl:
        x_raw, y_batch = x_raw.to(device), y_batch.to(device)
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f"Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}")
        base *= step

    if acc > best_acc:
        best_acc = acc
        trials = 0
        torch.save(model.state_dict(), "best.pth")
        print(f"Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}")
    else:
        trials += 1
        if trials >= patience:
            print(f"Early stopping on epoch {epoch}")
            break

print("Done!")

RuntimeError: Given groups=1, weight of size [16, 1, 3, 3, 3], expected input[256, 48, 1, 6, 6] to have 1 channels, but got 48 channels instead

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(loss_history, label='loss')
ax[0].set_title('Validation Loss History')
ax[0].set_xlabel('Epoch no.')
ax[0].set_ylabel('Loss')

ax[1].plot(smooth(acc_history, 5)[:-2], label='acc')
ax[1].set_title('Validation Accuracy History')
ax[1].set_xlabel('Epoch no.')
ax[1].set_ylabel('Accuracy');

In [ ]:
preds_array = np.array([])
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1).numpy()
        preds_array = np.concatenate((preds_array, preds), axis=None)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, preds_array)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2']
print(classification_report(y_test, preds_array, target_names=target_names))

In [ ]:
x_train[0].shape

In [ ]:
arr_2d= np.array([[1,2,3], [5,6,7], [9,10,11], [12,13,14]])
arr_2d.shape

In [ ]:
arr_2d

In [ ]:
arr_3d= arr_2d.reshape(2,2,3)
arr_3d[:,:,0], arr_3d[:,:,1], arr_3d[:,:,2]

In [ ]:
arr_3d[:,:,0].shape

In [ ]:
summary(model, input_size=(1, 37, 48)) 